# Update Regression Imitations Using StatsModels in Python

In **R** rolling regressions are quite easy to obtain.  Python APIs are trickier.  Here we demonstrate obtaining the equivalent to rolling regression output regressions using `statsmodels` basic regression routines.

The techniques here do *not* use Sherman-Morrison updates and are therefore quite unsuitable to treating high-frequency data.  Thus these examples have output that is numerically correct, but not computationally correct in cases where we desire the proper efficient calculations.

## Imports And Data

In [1]:
%matplotlib inline


In [2]:
import numpy as np
import scipy as sp
import statsmodels.regression.rolling as sm_roll_reg
import pandas as pd


In [3]:
import finm_teaching_utils  # You will use your own code

In [4]:
import rpy2.ipython

In [5]:
%load_ext rpy2.ipython

In [6]:
all_equity_prices = finm_teaching_utils.cached_fetch.fetch_quotemedia_prices()

Skipping any possible download of QUOTEMEDIA/PRICES
Reading CSV file /Users/brian/quandl_data_table_downloads/QUOTEMEDIA/PRICES_latest.zip
Sorting prices


In [7]:
_ddists = (all_equity_prices.index.get_level_values('Date')-pd.Timestamp('2023-01-01'))
equity_prices = all_equity_prices.loc[(_ddists<pd.Timedelta(days=30)) & (_ddists>=pd.Timedelta(days=-30))]

In [8]:
design_mx = equity_prices.loc[['SPY','SUN', 'XOM']].groupby(level='ticker').pct_change().unstack().transpose().dropna().loc['adj_close']
design_mx

ticker,SPY,SUN,XOM
Date,,,
2022-12-05,-0.017989,-0.020308,-0.027399
2022-12-06,-0.014415,0.018585,-0.027796
2022-12-07,-0.001701,0.015673,-0.002214
2022-12-08,0.007834,-0.010594,0.007429
2022-12-09,-0.007470,0.010009,-0.008428
2022-12-12,0.014417,0.009449,0.024628
2022-12-13,0.007570,0.008904,0.010934
2022-12-14,-0.006394,-0.009052,-0.007366
2022-12-15,-0.024462,0.000685,-0.009581


## BoxCar

In [9]:
import statsmodels.regression.linear_model as linreg

def boxcar_5_reg(_df):
    _p = linreg.OLS(_df['SUN'], _df[['SPY', 'XOM']]).fit(params_only=True).params
    return pd.DataFrame(data=[_p], index=[_df.index[-1]])

b_5_coeffs = pd.concat([boxcar_5_reg(design_mx.iloc[i-5:i]) for i in range(5, design_mx.shape[0])])
b_5_coeffs.iloc[:6]

,SPY,XOM
2022-12-09,0.417535,-0.352496
2022-12-12,-1.589001,0.607671
2022-12-13,-4.288105,2.969486
2022-12-14,-2.914380,2.206185
2022-12-15,-0.268692,0.547106
2022-12-16,0.269980,0.347744


### Checks

Here we check the results of the Dec 13 output against a completely independent implementation from **R**

In [10]:
test5 = design_mx.loc[ pd.Timestamp('2022-12-07') : pd.Timestamp('2022-12-13') ]
test5


ticker,SPY,SUN,XOM
Date,,,
2022-12-07,-0.001701,0.015673,-0.002214
2022-12-08,0.007834,-0.010594,0.007429
2022-12-09,-0.007470,0.010009,-0.008428
2022-12-12,0.014417,0.009449,0.024628
2022-12-13,0.007570,0.008904,0.010934


In [11]:
%%R -i test5

lm(SUN~SPY+XOM+0, data=test5)


Call:
lm(formula = SUN ~ SPY + XOM + 0, data = test5)

Coefficients:
   SPY     XOM  
-4.288   2.969  



## Exponential / Discounted

The `statsmodels` package does not have true discounted least squares.  So we use their weighted least squares with a weight set derived from (nontrivial) exponential weights we have manually generated.

In [12]:
half_life = 1.2
lma = 2**(-1./half_life)
threshold = 1. / (design_mx.shape[0]* 500)
print(f"Lambda: {lma}  -> Truncation threshold {threshold}")

Lambda: 0.5612310241546865  -> Truncation threshold 7.407407407407407e-05


In [13]:
w12 = lma**np.arange(design_mx.shape[0]-1, -1, -1)

# Cuts off weight values too small to care about, allowing use of naive routines without losing accuracy or having too many points
exp_1p2_wts = w12[ w12 > threshold]
print(f"""
Full weight at start and finish:\n{w12[[0,1, -2, -1]]} N={w12.shape[0]}

Truncated at start and finish:\n{exp_1p2_wts[[0,1, -2, -1]]}  N={exp_1p2_wts.shape[0]}
""")



Full weight at start and finish:
[3.00388586e-07 5.35231613e-07 5.61231024e-01 1.00000000e+00] N=27

Truncated at start and finish:
[9.68872712e-05 1.72633492e-04 5.61231024e-01 1.00000000e+00]  N=17



We will say there are "enough" points to go ahead and regress if we have at least got all weights bigger than 1/20

In [14]:
enough_points = np.where( w12 > 1./20 )[0].shape[0]
enough_points

6

In [15]:
import statsmodels.regression.linear_model as linreg

def exp_1_2_reg(_df):
    _n = min(exp_1p2_wts.shape[0], _df.shape[0])
    _ldf = _df.iloc[-_n:]
    _wt = exp_1p2_wts[-_n:]
    _p = linreg.WLS(_ldf['SUN'], _ldf[['SPY', 'XOM']], weights=_wt).fit(params_only=True).params
    return pd.DataFrame(data=[_p], index=[_df.index[-1]])

exp_1_2_coeffs = pd.concat([exp_1_2_reg(design_mx.iloc[max(0,i-exp_1p2_wts.shape[0]):i]) for i in range(enough_points, design_mx.shape[0])])
exp_1_2_coeffs.iloc[:6]

,SPY,XOM
2022-12-12,-3.222329,2.132432
2022-12-13,-1.763503,1.413680
2022-12-14,0.989619,-0.210738
2022-12-15,-0.256065,0.604255
2022-12-16,0.489067,0.209066
2022-12-19,1.202125,-1.022572


### Checks

Manually checking the first few, where we have somewhere between "enough" and the full complement of thresholded weights

In [16]:
exp_1_2_coeffs.loc[pd.Timestamp('2022-12-13')]

SPY   -1.763503
XOM    1.413680
Name: 2022-12-13 00:00:00, dtype: float64

In [17]:
first_few = design_mx.loc[:pd.Timestamp('2022-12-13')]
linreg.WLS(
    first_few['SUN'], 
    first_few[['SPY', 'XOM']], 
    weights=exp_1p2_wts[-first_few.shape[0]:]
).fit(params_only=True).params

SPY   -1.763503
XOM    1.413680
dtype: float64

We also check the fit quality in cases where our weight truncation has actually taken effect

In [18]:
exp_1_2_coeffs.iloc[-4:]

,SPY,XOM
2023-01-06,0.118861,0.437270
2023-01-09,0.161510,0.266407
2023-01-10,0.236741,0.368750
2023-01-11,0.234911,0.368473


In [19]:
many = design_mx.loc[:pd.Timestamp('2023-01-09')]
linreg.WLS(
    many['SUN'], 
    many[['SPY', 'XOM']], 
    weights=w12[-many.shape[0]:]
).fit(params_only=True).params


SPY    0.161507
XOM    0.266405
dtype: float64

These are satisfyingly close